# Milestone 5: Morphology helper

Setup

In [ ]:
import getpass
import logging
from pathlib import Path

from bluepysnap.api import Api

logging.basicConfig(level=logging.WARNING)

ORGANIZATION = "bbp"
PROJECT = "thalamus"

Paste the token from https://bbp.epfl.ch/nexus/web/

In [ ]:
TOKEN = getpass.getpass()

In [ ]:
nexus_config = "../../../examples/configurations/api/forge.yml"
api = Api(nexus_config=nexus_config, bucket=f"{ORGANIZATION}/{PROJECT}", token=TOKEN, debug=False)

### From a KG URL retrieve a Morphology Collection

In [ ]:
morphology_release_id = "https://bbp.epfl.ch/neurosciencegraph/data/20191208"
morphology_release = api.get_entity_by_id(morphology_release_id)
print(morphology_release)

### Get information related to a Morphology Collection

In [ ]:
print("Description:", morphology_release.description)
print("Creator:", morphology_release.createdBy)
print("Creation date:", morphology_release.createdAt)
print("Project:", morphology_release.project)
print("Species:", morphology_release.species.label)
print("Brain region:", morphology_release.brainLocation.brainRegion.label)

### Retrieve and access a nested resource

In [ ]:
print(morphology_release.morphologyIndex.distribution.url)

### Instantiate a MorphDB object from a Morphology Collection and a unique identifier

In [ ]:
print(morphology_release.instance)

In [ ]:
morphology_release.instance.df

In [ ]:
morphology_release.instance.df.iloc[0]

In [ ]:
import neurom
from neurom import viewer

path = morphology_release.instance.df.iloc[0]["path"]
nrn = neurom.load_neuron(path)
fig, ax = viewer.draw(nrn, mode="3d")